In [ ]:
import logging
from neural_poisson.data.shapenet import ShapeNetCoreDataset, log
from pathlib import Path
import time

import lightning as L
log.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
log.addHandler(handler)

def measure_dataset(dataset, N=100):
    start_time = time.time()
    for i in range(N):
        batch = dataset[i]
    end_time = time.time()
    duration = ((end_time - start_time) / N) * 1000
    print(f"Time taken for 100 iterations (GPU): {duration:.6f} ms")

# define the paths
model_id = "107bce22d72f322eedf1bb0b62653056"
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCore/04256520"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

In [ ]:
s = time.time()
dataset = ShapeNetCoreDataset(
    path=shapenet_path,
    dist=2.0,
    fov=30.0,
    image_size=256,
    fill_depth="max2",
    segments=8,   # subsample x4
    resolution=0.001,
    num_chunks=100,
    chunk_size=50_000,
    max_empty_points=0,
    max_close_points=0,
    max_surface_points=100_000,
    vector_field_mode="nearest_neighbor",
    sigma=0.001,
)
print(f"Time taken for creation: {((time.time() - s)  * 1000):.6f} ms")

In [ ]:
dataset.plot_point_cloud("vectors_surface")

In [ ]:
dataset.plot_point_cloud("vectors_close")

In [ ]:
dataset.plot_normal_maps(False)

In [ ]:
from neural_poisson.model.neural_poisson import NeuralPoisson
import open3d as o3d
from neural_poisson.data.prepare import load_mesh_o3d, to_pcd_o3d, to_mesh_o3d

epoch = 10
run_path = Path("/home/borth/2d-gaussian-splatting/logs/2025-02-26/09-27-46")
ckpt_path = run_path / f"checkpoints/epoch_{epoch:05}.ckpt"
mesh_path = run_path / f"mesh/epoch_{epoch:05}.obj"

model = NeuralPoisson.load_from_checkpoint(ckpt_path)
mesh_model = to_mesh_o3d(model.to_mesh())
mesh_ckpt = load_mesh_o3d(mesh_path)
mesh_gt = load_mesh_o3d(shapenet_path)
# pcd = to_pcd_o3d(dataset.points_surface, dataset.normals_surface)


In [ ]:
print(mesh_model)
print(mesh_ckpt)
o3d.visualization.draw_plotly([mesh_ckpt, mesh_gt])

In [ ]:
import torch
from pytorch3d.ops.marching_cubes import marching_cubes
from pytorch3d.structures import Meshes
from neural_poisson.data.grid import coord_grid

def to_mesh(model, voxel_size: int | None = None):
    # prepare the evaluation
    model.eval()

    # fetch the point on the grid lattice
    N = model.hparams["voxel_size"] if voxel_size is None else voxel_size
    grid = coord_grid(voxel_size=N, domain=model.hparams["domain"]).reshape(-1, 3)

    # evaluate the indicator function on the grid structure
    sdfs = []
    for points in torch.split(grid, model.hparams["chunk_size"]):
        x, _ = model.forward(points.to(model.device))
        # convert indicator to "sdf" value, where negative is inside
        sdfs.append(-x.detach().cpu())
    sdf_grid = torch.cat(sdfs).reshape(N, N, N)
    sdf_grid = sdf_grid.permute(2, 1, 0)

    # log the slice of the mesh
    # for dim in ["x", "y", "z"]:
    #     self.log_video(sdf_grid=sdf_grid, dim=dim)

    # ensures that we have a valid isolevel and can extract a mesh
    isolevel = model.isolevel
    if isolevel > sdf_grid.max() or isolevel < sdf_grid.min():
        isolevel = (sdf_grid.max().item() - sdf_grid.min().item()) / 2

    # perform marching cubes
    verts, faces = marching_cubes(sdf_grid[None], isolevel=isolevel)
    if not len(verts[0]):
        return None
    return Meshes(verts=verts, faces=faces).to(model.device)

mesh_dev = to_mesh_o3d(to_mesh(model))
o3d.visualization.draw_plotly([mesh_dev, mesh_gt])

In [ ]:
o3d.visualization.draw_plotly([mesh_dev])

In [ ]:

path = "/home/borth/2d-gaussian-splatting/logs/2025-02-26/08-49-28/mesh/epoch_00030.obj"
pcd = to_pcd_o3d(dataset.points_surface, dataset.normals_surface)
mesh = load_mesh_o3d(path)
mesh_gt = load_mesh_o3d(shapenet_path)
o3d.visualization.draw_plotly([mesh_gt, mesh, pcd])


In [ ]:
model.isolevel = 0.25
verts, faces = model.to_mesh()
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts.cpu().numpy())
mesh.triangles = o3d.utility.Vector3iVector(faces.cpu().numpy())

In [ ]:
mesh.compute_vertex_normals()
o3d.visualization.draw_plotly([mesh])

In [ ]:
import torch
from pytorch3d.structures import Meshes
from neural_poisson.data.prepare import get_depth_camera_space_attributes, rasterize_attributes
import matplotlib.pyplot as plt

# Assuming verts and faces are tensors
# verts: (V, 3) tensor of vertex positions
# faces: (F, 3) tensor of triangle indices

device = "cuda" if torch.cuda.is_available() else "cpu"

# Example: Move tensors to the same device
verts = verts.to(device)
faces = faces.to(device)

# Create a PyTorch3D Mesh
mesh = Meshes(verts=[verts], faces=[faces])
mesh = dataset.mesh

# camera = dataset.cameras[9]
for camera in dataset.cameras:

    attributes = get_depth_camera_space_attributes(mesh=mesh, camera=camera)
    depth_map, mask = rasterize_attributes(
        mesh=mesh,
        camera=camera,
        attributes=attributes,
        image_size=256,
    )
    plt.imshow(depth_map[0].detach().cpu().numpy())